# IN2110 Oblig 2a
## Oppgave 2 og 3
Etter at du har trent modellen fra kommandolinja, gjøres resten av programmeringen, rapporteringen og besvarelse ellers i dette dokumentet. Lag celler for markdown og kode etter behov.

In [3]:
import spacy
from helpers.conllu import ConlluDoc
print("spaCy version:",spacy.__version__) 

spaCy version: 3.7.4


In [4]:
nb = spacy.load("model-best")
conllu_dev = ConlluDoc.from_file("no_bokmaal-ud-dev.conllu")
dev_docs = conllu_dev.to_spacy(nb)
dev_docs_unlabeled = conllu_dev.to_spacy(nb, keep_labels=False)

In [75]:
import numpy as np
def parse(dok_list):
    parsed=[]
    for d in dok_list:
        pd = nb(d)
        parsed.append(pd)
    return parsed

    

In [90]:



def attachment_score(true, pred):
    """Tar inn to lister av doc der liste nr 2 har predikert depedens. Finner andel korrekte head (UAS), og korrekte head+dependensrelasjoner (LAS)
    Merk at depedensrelasjonene som SpaCy estimerer har "ROOT" med store bokstaver, mens våre data har "root" med små.
    Dette er den samme relasjonen, og skal teller som korrekt estimert """
    #TODO
    uas_sentence=[]
    las_sentence=[]
    for i in range(len(pred)):
        #Finner antall ord i setningen og teller antall riktige hoder, samt riktige hoder og avhengigheter.
        num_words =len(pred[i])
        correct_head =0
        c_head_dep =0
    
        for j in range(len(pred[i])):
            if pred[i][j].head.text==true[i][j].head.text:
                correct_head+=1
                if pred[i][j].dep_.lower==true[i][j].dep_.lower:
                    c_head_dep+=1
        
        #UAS og LAS for hver setning i datasettet
        las_sentence.append(c_head_dep/num_words)
        uas_sentence.append(correct_head/num_words)

    #regner ut gjennomsnittlig LAS/UAS for alle setninger.
    uas = sum(uas_sentence)/len(uas_sentence)
    las=sum(las_sentence)/len(las_sentence)
    return uas, las



In [89]:
#UAS og LAS på dev settet.
uas,las =attachment_score(dev_docs,parse(dev_docs_unlabeled))


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
27 19
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
17 11
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
21 14
True
True
True
True
True
5 5
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
39 23
True
True
True
True
True
True
True
True
True
9 6
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
16 13
True
True
True
True
True
5 3
True
True
True
True
True
True
True
7 6
True
True
True
True
True
True
True
True
8 4
True
True
True
True
True
True
True
True
True
True
True
True
12 9
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


UAS for dev settet ligger på ca 90%. Den gir altså et godt resultat for riktig predikert hode. 
LAS er nokså mye lavere, ca 62%. Dette forteller oss at modellen er god til å predikere riktig hode, men ikke fult så god til å predikere både riktig hode og dependence-relasjon. 
Det gir likevel mening at LAS gir en noe mindre score, ettersom at modellen her må ha predikert begge riktig. 

In [92]:
#Evaluering nynorsk
conllu_dev = ConlluDoc.from_file("no_nynorsk-ud-dev.conllu")
dev_docs = conllu_dev.to_spacy(nb)
dev_docs_unlabeled = conllu_dev.to_spacy(nb, keep_labels=False)
uas_nn,las_nn =attachment_score(dev_docs,parse(dev_docs_unlabeled))


In [93]:
conllu_dev = ConlluDoc.from_file("no_nynorsklia-ud-dev.conllu")
dev_docs = conllu_dev.to_spacy(nb)
dev_docs_unlabeled = conllu_dev.to_spacy(nb, keep_labels=False)

uas_nnl,las_nnl=attachment_score(dev_docs,parse(dev_docs_unlabeled))

In [96]:
#Tabell for de forskjellige datasettene.
#Rad 1:UAS, rad 2: LAS.
import pandas as pd
resultater ={"Bokmål":[uas,las],"Nynorsk":[uas_nn,las_nn],"nonorsklia":[uas_nnl,las_nnl]}
df =pd.DataFrame(resultater)
df

,Bokmål,Nynorsk,nonorsklia
0,0.896958,0.700869,0.581628
1,0.627721,0.420841,0.305691


Generelt kan en observere at LAS og UAS er lavere for nynorsk-settet og enda lavere for det transkriberte lyd-settet. I likhet med resulttatene er UAS høyere enn Las for begge settene. 
Disse resultatene gir mening ettersom at modellen er trent på tekster skrevet på bokmål. Den vil dermed få litt andre resultater for nynorsk, og vil variere enda mer for transkribert tale, ettersom folk gjerne har en annen setningsstruktur når de uttrykker seg muntlig.